# Improvements to make:

1. Calculate terms in the continuity equation.
2. Look at some numbers from NON snow dates - don't need to worry about instrument height.
3. Duplicate/interpolate density (and mixing ratio) measurements across the towers in a more sophisticated way (currently using ground-relative height, should be using snow-surface-relative height).
4. Looking at moisture/water vapor data at downvalley sites - KPS annex site? Brush Creek

T should be in kelvin

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import altair as alt
alt.data_transformers.enable('json')
alt.renderers.enable('jupyterlab')
from sublimpy import tidy, utils
import pytz
import datetime as dt

seconds_in_timestep = 60*30
from metpy.constants import density_water

from metpy.units import units
import pint_pandas
from metpy import constants
from scipy import interpolate
np.set_printoptions(suppress=True,precision=10)

# Parameters for calculations

In [ ]:
HEIGHTS = [1,3,10]
HORIZ_GRID_SPACING = 50
VERT_GRID_SPACING = 20
start_date = '20221130'
end_date = '20230509'
# start_date = '20221101'
# end_date = '20230619'

data_cutoff_date = '20230508'
# data_cutoff_date = '20230619'

    # streamwise-coords
# tidy_df = pd.read_parquet(f"tidy_df_{start_date}_{end_date}_planar_fit_multiplane.parquet")
    # slope-adjusted earthwise-coords
tidy_df = pd.read_parquet(f"tidy_df_{start_date}_{end_date}_planar_fit.parquet")
    # sonic coords
# tidy_df = pd.read_parquet(f"tidy_df_{start_date}_{end_date}_noplanar_fit.parquet")

# method_numerical_advection = 'wind_divergence'      # q * d/dx_i (u_i)
# method_numerical_advection = 'scalar_divergence'    # u_i * d/dx_i (q)
# method_numerical_advection = 'summed'               # q * d/dx_i (u_i) + u_i * d/dx_i (q)
# method_numerical_advection = 'derivative'           # d/dx_i (q * u_i)

In [ ]:
def daily_w_cycle(src, title, ylim):
    return alt.Chart(
        src[src.variable.isin([
            'w_2m_c', 'w_3m_c', 'w_5m_c', 'w_10m_c', 'w_15m_c', 'w_20m_c',
            'w_3m_ue','w_10m_ue',
            'w_3m_uw','w_10m_uw',
            'w_3m_d','w_10m_d',
        ])]
    ).mark_line().encode(
        alt.X('hoursminutes(time):T'),
        alt.Y('median(value):Q').title('Wind speed (m/s)').scale(domain=ylim),
        alt.Color('height:O').scale(scheme='sinebow'),
        alt.StrokeDash('tower:N'),
        tooltip='variable',
    ).properties(width = 200, height = 200, title=title)

(

    daily_w_cycle(
        pd.read_parquet(f"tidy_df_{start_date}_{end_date}_planar_fit.parquet"),
        'SRLEC',
        ylim=[-0.10, 0.05]
    ) | daily_w_cycle(
        pd.read_parquet(f"tidy_df_{start_date}_{end_date}_noplanar_fit.parquet"),
        'Geographic',
        ylim=[-0.10, 0.05]
    ) | 
    daily_w_cycle(
        pd.read_parquet(f"tidy_df_{start_date}_{end_date}_planar_fit_multiplane.parquet"),
        'Streamwise',
        ylim=[-0.10, 0.05]
    )
).properties(title='Vertical wind speed, different coordinate systems').display(renderer='svg')

In [63]:
src = pd.read_parquet(f"tidy_df_{start_date}_{end_date}_planar_fit_multiplane.parquet")
src = utils.modify_df_timezone(src, 'UTC', 'US/Mountain')
alt.Chart(
        src[src.variable.isin([
            'w_2m_c', 'w_3m_c', 'w_5m_c', 'w_10m_c', 'w_15m_c', 'w_20m_c',
            'w_3m_ue','w_10m_ue',
            'w_3m_uw','w_10m_uw',
            'w_3m_d','w_10m_d',
        ])]
    ).transform_window(
        rolling_avg = "mean(value)",
        frame=[-2,2],
        groupby=['height', 'tower', 'variable']
    ).mark_line().encode(
        alt.X('hoursminutes(time):T'),
        alt.Y('mean(rolling_avg):Q').title('Wind speed (m/s)').scale(domain=[-0.02, 0.02]),
        alt.Color('tower:N'),
        alt.Facet('height:O', columns=3),
        tooltip='variable',
    ).properties(width = 100, height = 100, title=    'Streamwise',).display(renderer='svg')

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [65]:
src = pd.read_parquet(f"tidy_df_{start_date}_{end_date}_planar_fit_multiplane.parquet")
src = utils.modify_df_timezone(src, 'UTC', 'US/Mountain')
alt.Chart(
        src[src.variable.isin([
            'dir_2m_c', 'dir_3m_c', 'dir_5m_c', 'dir_10m_c', 'dir_15m_c', 'dir_20m_c',
            'dir_3m_ue','dir_10m_ue',
            'dir_3m_uw','dir_10m_uw',
            'dir_3m_d','dir_10m_d',
        ])]
    ).transform_window(
        rolling_avg = "median(value)",
        frame=[-1,1],
        groupby=['height', 'tower', 'variable']
    ).mark_line().encode(
        alt.X('hoursminutes(time):T'),
        alt.Y('median(rolling_avg):Q').title('Wind speed (m/s)').scale(domain=[0, 360]),
        alt.Color('height:O'),
        alt.Facet('tower:N', columns=2),
        tooltip='variable',
    ).properties(width = 200, height = 200, title=    'Streamwise',).display(renderer='svg')

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [ ]:
src = pd.read_parquet(f"tidy_df_20221101_20230619_noplanar_fit.parquet")
alt.Chart(
        src[src.variable.isin([
            'w_2m_c', 'w_3m_c', 'w_5m_c', 'w_10m_c', 'w_15m_c', 'w_20m_c',
            'w_3m_ue','w_10m_ue',
            'w_3m_uw','w_10m_uw',
            'w_3m_d','w_10m_d',
        ])]
    ).mark_line().encode(
        alt.X('hoursminutes(time):T'),
        alt.Y('mean(value):Q').title('Wind speed (m/s)'),
        alt.Color('height:O').scale(scheme='sinebow'),
        alt.StrokeDash('tower:N'),
        alt.Facet('month(time):T', columns=4).sort([11,12,1,2,3,4,5,6]),
        tooltip='variable',
    ).properties(width = 200, height = 200)

# Prepare data

## Open SOS Measurement Dataset

In [ ]:
# convert time column to datetime
tidy_df['time'] = pd.to_datetime(tidy_df['time'])
tidy_df = utils.modify_df_timezone(tidy_df, pytz.UTC, 'US/Mountain')
# limit data to our dates of interest, based on continuous snow cover at Kettle Ponds
tidy_df = tidy_df.set_index('time').sort_index().loc[start_date:data_cutoff_date].reset_index()

In [ ]:
## Add combined blowing snow flux variable
tidy_df = tidy.tidy_df_add_variable(
    tidy_df,
    (
        tidy_df.query("variable == 'SF_avg_1m_ue'")['value'].values + 
        tidy_df.query("variable == 'SF_avg_2m_ue'")['value'].values
    ), 
    'SF_avg_ue', 'snow flux', 1, 'ue',
)

## Add absolute humidity measurements by converting hygrometer measurements

In [ ]:
tower_height_keys = tidy_df[tidy_df.measurement=='specific humidity'].groupby(['tower', 'height']).indices.keys()
for t, h in tower_height_keys:
    this_tower_height_tidy_df = tidy_df.query(
            f"tower == '{t}'"
        ).query(
            f"height == {h}"
        )
    specific_humidity_values = this_tower_height_tidy_df.query(
            "measurement == 'specific humidity'"
        ).set_index('time')[['value']].rename(columns={'value': 'specific humidity'})
    air_density_values = this_tower_height_tidy_df.query(
            "measurement == 'air density'"
        ).set_index('time')[['value']].rename(columns={'value': 'air density'})
    combined_df = specific_humidity_values.join(air_density_values)

    abs_humidity_values = (
        combined_df['specific humidity'].values * units('g/g')
    ).to('g/kg') * (
        combined_df['air density'].values * units('kg/m^3')
    ).m

    tidy_df = tidy.tidy_df_add_variable(
        tidy_df,
        abs_humidity_values,
        f"absolutehumidity_{int(h)}m_{t}",
        'absolute humidity',
        int(h),
        t
    )

## Calibrate gas analyzer measurements

We calibrate by assuming that all gas analyzers have the same seasonal mean as the corresponding hygrometer measurement on the central tower (at a given height)

### With seasonal mean

In [ ]:
hygrometer_absolute_humidity_mean = (
    1000 * tidy_df[tidy_df.measurement=='specific humidity'].groupby(['tower', 'height'])[['value']].mean() *\
    tidy_df[tidy_df.measurement=='air density'].groupby(['tower', 'height'])[['value']].mean()
).reset_index().query("tower == 'c'")

In [ ]:
ec_absolute_humidity_mean = tidy_df[tidy_df.measurement=='Water vapor density'].groupby(['variable', 'tower', 'height'])[['value']].mean().reset_index()

In [ ]:
corrections_df = ec_absolute_humidity_mean.merge(
    hygrometer_absolute_humidity_mean[['height', 'value']].rename(columns={'value': 'truth'}),
    on='height'
)
corrections_df['offset'] = corrections_df['value'] - corrections_df['truth']
corrections_df

Update dataset with corrections

In [ ]:
src = tidy_df[tidy_df.measurement=='Water vapor density']
src = src[src.height.isin([1,3,10])]
src

In [ ]:
alt.Chart(
    (
    1000 * tidy_df[tidy_df.measurement=='specific humidity'].groupby(['tower', 'height'])[['value']].mean() *\
    tidy_df[tidy_df.measurement=='air density'].groupby(['tower', 'height'])[['value']].mean()
    ).reset_index()
).mark_point(shape='square', filled=True, color='black', size=20).encode(
    alt.X("value:Q"),
    alt.Y("height:Q")
).properties(width=150, height = 150)\
+ alt.Chart(
    tidy_df[tidy_df.measurement=='Water vapor density'].groupby(['variable', 'tower', 'height'])[['value']].mean().reset_index()
).mark_circle(size=40).encode(
    alt.X("value:Q"),
    alt.Y("height:Q"),
    alt.Color('tower:N')
).properties(width=150, height = 150)

In [ ]:


alt.Chart(
    src[ src.time > '20221212' ][ src.time < '20221214' ]
).mark_line().encode(
    alt.X("time:T"),
    alt.Y("value:Q"),
    alt.Color("height:N"),
    detail='variable'
)

In [ ]:
for idx, row in corrections_df.iterrows():
    src = tidy_df.query(f"variable == '{row['variable']}'")
    src = src.assign(value = src.value - row['offset'])
    tidy_df = tidy_df[tidy_df.variable != row['variable']]
    tidy_df = pd.concat([tidy_df, src])

In [ ]:
alt.Chart(
    (
    1000 * tidy_df[tidy_df.measurement=='specific humidity'].groupby(['tower', 'height'])[['value']].mean() *\
    tidy_df[tidy_df.measurement=='air density'].groupby(['tower', 'height'])[['value']].mean()
    ).reset_index()
).mark_point(shape='square', filled=True, color='black', size=20).encode(
    alt.X("value:Q"),
    alt.Y("height:Q")
).properties(width=150, height = 150)\
+ alt.Chart(
    tidy_df[tidy_df.measurement=='Water vapor density'].groupby(['variable', 'tower', 'height'])[['value']].mean().reset_index()
).mark_circle(size=40).encode(
    alt.X("value:Q"),
    alt.Y("height:Q"),
    alt.Color('tower:N')
).properties(width=150, height = 150)

In [ ]:
src = tidy_df[tidy_df.measurement=='Water vapor density']
src = src[src.height.isin([1,3,10])]
abs_hum = alt.Chart(
    src[ src.time > '20221212' ][ src.time < '20221214' ]
).mark_line(strokeWidth=0.5).encode(
    alt.X("time:T"),
    alt.Y("value:Q").title("Absolute humidity (g/m^3)").scale(zero=False),
    alt.Color("height:N"),
    alt.Shape('tower:N'),
    detail='variable'
).properties(width=600)

src = tidy_df[tidy_df.measurement=='snow depth']
snowdepth = alt.Chart(
    src[ src.time > '20221212' ][ src.time < '20221214' ]
).mark_line(strokeWidth=0.5).encode(
    alt.X("time:T"),
    alt.Y("value:Q").title("Snow depth (m)"),
    alt.Shape('tower:N'),
    detail='variable'
).properties(width=600, height=150)

(snowdepth & abs_hum).resolve_scale(color='independent', shape='independent')

### With monthly means

In [ ]:
# filtered_spechumidity = tidy_df[tidy_df.measurement=='specific humidity']
# hygrometer_absolute_humidity_mean = (
#     1000 * filtered_spechumidity.groupby(['tower', 'height', filtered_spechumidity.time.dt.month])[['value']].mean() *\
#     tidy_df[tidy_df.measurement=='air density'].groupby(['tower', 'height'])[['value']].mean()
# ).reset_index().query("tower == 'c'")

In [ ]:
# filtered_abshumidity = tidy_df[tidy_df.measurement=='Water vapor density']
# ec_absolute_humidity_mean = filtered_abshumidity.groupby([
#     'variable', 'tower', 'height', filtered_abshumidity.time.dt.month
# ])[['value']].mean().reset_index()
# ec_absolute_humidity_mean

In [ ]:
# corrections_df = ec_absolute_humidity_mean.merge(
#     hygrometer_absolute_humidity_mean[['height', 'value', 'time', 'tower']].rename(columns={'value': 'truth'}),
#     on=['height', 'tower', 'time']
# )
# corrections_df['offset'] = corrections_df['value'] - corrections_df['truth']


Update dataset with corrections

In [ ]:
# corrected_measurements = []
# for variable in corrections_df.variable.unique():
#     for month in corrections_df[corrections_df.variable == variable].time.unique():
#         src = tidy_df.query(f"variable == '{variable}'")
#         src = src[src.time.dt.month == month]
#         row = corrections_df.set_index(['variable', 'time']).loc[variable, month]
#         src = src.assign(value = src.value - row['offset'])
#         corrected_measurements.append(src)

In [ ]:
# for variable in corrections_df.variable.unique():
#     tidy_df = tidy_df[tidy_df.variable != variable]
# tidy_df = pd.concat([tidy_df] + corrected_measurements)

In [ ]:
# hygr_vals = (
#     1000 * tidy_df[tidy_df.measurement=='specific humidity'].groupby(['tower', 'height'])[['value']].mean() *\
#     tidy_df[tidy_df.measurement=='air density'].groupby(['tower', 'height'])[['value']].mean()
#     ).reset_index()
# irga_vals = tidy_df[tidy_df.measurement=='Water vapor density'].groupby(['variable', 'tower', 'height'])[['value']].mean().reset_index()
# hygr_vals = hygr_vals[hygr_vals.time.dt.month==12]
# irga_vals = irga_vals[irga_vals.time.dt.month==12]
# alt.Chart(hygr_vals).mark_point(shape='square', filled=True, color='black', size=20).encode(
#     alt.X("value:Q"),
#     alt.Y("height:Q")
# ).properties(width=150, height = 150)\
# + alt.Chart(irga_vals).mark_circle(size=40).encode(
#     alt.X("value:Q"),
#     alt.Y("height:Q"),
#     alt.Color('tower:N')
# ).properties(width=150, height = 150)

# Identify categories for timestamps

In [ ]:
# Identify lists of timestamps for different categories
bs_times = tidy_df.query("variable == 'SF_avg_ue'").query("value > 0").time
nobs_times = tidy_df.query("variable == 'SF_avg_ue'").query("value == 0").time

decoupled_times = tidy_df.query("variable == 'omega_3m_c'").query("value < 0.43").time
weaklycoupled_times = tidy_df.query("variable == 'omega_3m_c'").query("value >= 0.43").query("value <= 0.61").time
coupled_times = tidy_df.query("variable == 'omega_3m_c'").query("value > 0.61").time

ri_stable_times = tidy_df.query("variable == 'Ri_3m_c'").query("value > 0.25").time
ri_unstable_times = tidy_df.query("variable == 'Ri_3m_c'").query("value < -0.01").time
ri_neutral_times = tidy_df.query("variable == 'Ri_3m_c'").query("value >= -0.01").query("value <= 0.25").time

tgrad_stable_times = tidy_df.query("variable == 'temp_gradient_3m_c'").query("value > 0.01").time
tgrad_unstable_times = tidy_df.query("variable == 'temp_gradient_3m_c'").query("value < -0.01").time
tgrad_neutral_times = tidy_df.query("variable == 'temp_gradient_3m_c'").query("value >= -0.01").query("value <= 0.01").time

december_times = tidy_df[tidy_df.time.dt.month == 12].time
january_times = tidy_df[tidy_df.time.dt.month == 1].time
february_times = tidy_df[tidy_df.time.dt.month == 2].time
march_times = tidy_df[tidy_df.time.dt.month == 3].time
april_times = tidy_df[tidy_df.time.dt.month == 4].time

midwinter_times = tidy_df[tidy_df.time < '20230320'].time
spring_times = tidy_df[tidy_df.time > '20230320'].time

In [ ]:
precip_df = xr.open_dataset("/Users/elischwat/Development/data/sublimationofsnow/precip_danny/precipitation_rate_gts_w23.nc")['corrected_prcp_rate_m2'].to_dataframe()

is_snowing_dates = pd.concat([
    precip_df.query("corrected_prcp_rate_m2 > 0").index.to_series(),
    precip_df.query("corrected_prcp_rate_m2 > 0").index.to_series() + dt.timedelta(minutes=30)
])
    
is_not_snowing_dates = pd.concat([
    precip_df.query("corrected_prcp_rate_m2 <= 0").index.to_series(),
    precip_df.query("corrected_prcp_rate_m2 <= 0").index.to_series() + dt.timedelta(minutes=30)
])

# Create tables

## Instrument location info (georeferenced)
We use a file with theodolite/GPS readings provided by NCAR. 

In [ ]:
instrument_loc_df = pd.read_csv("~/Development/data/sublimationofsnow/SOSm.txt", names = ['ec', 'x', 'y', 'z'])
instrument_loc_df = instrument_loc_df[ 
    instrument_loc_df['ec'].str.startswith('CS')
    |
    instrument_loc_df['ec'].str.startswith('DS') 
    |
    instrument_loc_df['ec'].str.startswith('UWS') 
    |
    instrument_loc_df['ec'].str.startswith('UES') 
]
instrument_loc_df = instrument_loc_df[ 
    instrument_loc_df['ec'].str.endswith('T') 
    |
    instrument_loc_df['ec'].str.endswith('B') 
]
instrument_loc_df['top or bottom'] = instrument_loc_df['ec'].str[-1]
instrument_loc_df['tower'] = instrument_loc_df['ec'].apply(lambda str: str.split('S')[0].lower())
instrument_loc_df['height'] = instrument_loc_df['ec'].apply(lambda str: int(str.split('S')[1][:-1]))
instrument_loc_df = instrument_loc_df.drop(columns='ec')
instrument_loc_df = instrument_loc_df.pivot(index=['height', 'tower'], columns='top or bottom').reset_index()
instrument_loc_df = instrument_loc_df.set_index(['height', 'tower']).groupby(level=0, axis=1).mean()
instrument_loc_df

In [ ]:
import geopandas as gpd
gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(
        instrument_loc_df.loc[3].x,
        instrument_loc_df.loc[3].y
    ),
    crs='EPSG:32613'
).to_file("~/Downloads/tower_locs.geojson")

## Wind field measurements

In [ ]:
wind_field_df = tidy_df[tidy_df.measurement.isin(['u','v','w']) & tidy_df.height.isin(HEIGHTS)]
wind_field_df = round(wind_field_df.pivot_table(index='time', columns=['height', 'tower', 'measurement'], values='value'), 4)
wind_field_df

## Turbulent water vapor flux measurements

In [ ]:
turb_flux_field_df = tidy_df[tidy_df.measurement.isin(['u_h2o_','v_h2o_','w_h2o_']) & tidy_df.height.isin(HEIGHTS)]
turb_flux_field_df = round(turb_flux_field_df.pivot_table(index='time', columns=['height', 'tower', 'measurement'], values='value'), 4)
turb_flux_field_df

## Turbulent temperature flux measurements

In [ ]:
temp_turb_flux_field_df = tidy_df[tidy_df.measurement.isin(['u_tc_','v_tc_','w_tc_']) & tidy_df.height.isin(HEIGHTS)]
temp_turb_flux_field_df = round(temp_turb_flux_field_df.pivot_table(index='time', columns=['height', 'tower', 'measurement'], values='value'), 4)
temp_turb_flux_field_df

## Humidity measurements

In [ ]:
abs_hum_field_df = tidy_df[tidy_df.measurement.isin(['Water vapor density']) & tidy_df.height.isin(HEIGHTS)]
abs_hum_field_df.measurement = 'q'
abs_hum_field_df = round(
    abs_hum_field_df.pivot_table(
        index='time', columns=['height', 'tower', 'measurement'], values='value'
    ), 
    4
)
abs_hum_field_df

## Advective flux measurements

In [ ]:
ls = []
for h in wind_field_df.columns.get_level_values('height').unique():
    for t in wind_field_df.columns.get_level_values('tower').unique():
        this_wind_df = wind_field_df[(h,t)].copy()
        this_abs_hum_df = abs_hum_field_df[(h,t)].copy()  
        this_wind_df['uq'] = this_wind_df['u']*this_abs_hum_df['q']
        this_wind_df['vq'] = this_wind_df['v']*this_abs_hum_df['q']
        this_wind_df['wq'] = this_wind_df['w']*this_abs_hum_df['q']
        new = pd.concat([this_wind_df], axis=1, keys=[(h,t)])
        ls.append(new.drop(columns=[(h,t,'u'),(h,t,'v'),(h,t,'w')]))

adv_flux_field_df = ls[0]
for l in ls[1:]:
    adv_flux_field_df = adv_flux_field_df.join(l)
adv_flux_field_df.columns = adv_flux_field_df.columns.set_names('height', level=0)
adv_flux_field_df.columns = adv_flux_field_df.columns.set_names('tower', level=1)
adv_flux_field_df

## Dry air density measurements

In [ ]:
# gather dry air density measurements
dryair_density_field_df = tidy_df[tidy_df.measurement.isin(['dry air density']) & tidy_df.height.isin(HEIGHTS)]
dryair_density_field_df.measurement = 'rho'
dryair_density_field_df = round(
    dryair_density_field_df.pivot_table(
        index='time', columns=['height', 'tower', 'measurement'], values='value'
    ), 
    4
)

# duplicate the dry air density measurements across the towers (THIS IS NAIVE)
dryair_density_for_tower_d = dryair_density_field_df.copy()
dryair_density_for_tower_d.columns = pd.MultiIndex.from_tuples([(cs[0], 'd', cs[2]) for cs in dryair_density_for_tower_d.columns])

dryair_density_for_tower_uw = dryair_density_field_df.copy()
dryair_density_for_tower_uw.columns = pd.MultiIndex.from_tuples([(cs[0], 'uw', cs[2]) for cs in dryair_density_for_tower_d.columns])

dryair_density_for_tower_ue = dryair_density_field_df.copy()
dryair_density_for_tower_ue.columns = pd.MultiIndex.from_tuples([(cs[0], 'ue', cs[2]) for cs in dryair_density_for_tower_d.columns])

dryair_density_field_df = dryair_density_field_df.join(
    dryair_density_for_tower_d
).join(
    dryair_density_for_tower_ue
).join(
    dryair_density_for_tower_uw
)

dryair_density_field_df.columns = dryair_density_field_df.columns.set_names(['height', 'tower', 'measurement'])

# convert from kg/m^3 to g/m^3
dryair_density_field_df = dryair_density_field_df*1000

dryair_density_field_df

# Temperature measurements

In [ ]:
# gather dry air density measurements
temp_field_df = tidy_df[tidy_df.measurement.isin(['temperature']) & tidy_df.height.isin(HEIGHTS)]
temp_field_df.measurement = 'T'
temp_field_df = round(
    temp_field_df.pivot_table(
        index='time', columns=['height', 'tower', 'measurement'], values='value'
    ), 
    4
)

# duplicate the dry air density measurements across the towers (THIS IS NAIVE)
temp_for_tower_d = temp_field_df.copy()
temp_for_tower_d.columns = pd.MultiIndex.from_tuples([(cs[0], 'd', cs[2]) for cs in temp_for_tower_d.columns])

temp_for_tower_uw = temp_field_df.copy()
temp_for_tower_uw.columns = pd.MultiIndex.from_tuples([(cs[0], 'uw', cs[2]) for cs in temp_for_tower_d.columns])

temp_for_tower_ue = temp_field_df.copy()
temp_for_tower_ue.columns = pd.MultiIndex.from_tuples([(cs[0], 'ue', cs[2]) for cs in temp_for_tower_d.columns])

temp_field_df = temp_field_df.join(
    temp_for_tower_d
).join(
    temp_for_tower_ue
).join(
    temp_for_tower_uw
)

temp_field_df.columns = temp_field_df.columns.set_names(['height', 'tower', 'measurement'])

temp_field_df

## Mixing ratio measurements

In [ ]:
mixing_ratio_field_df = abs_hum_field_df.droplevel(2, 1) / dryair_density_field_df.droplevel(2, 1)

mixing_ratio_field_df.columns = pd.MultiIndex.from_product(mixing_ratio_field_df.columns.levels + [['r']])
mixing_ratio_field_df.columns = mixing_ratio_field_df.columns.set_names('measurement', level=2)

mixing_ratio_field_df

In [ ]:
mixing_ratio_field_df.to_pickle('mixing_ratio_field_df.pkl')

# Divergence calculations

In [ ]:
def divergence(f,sp):
    """ 
    From: https://stackoverflow.com/a/67971515
    Computes divergence of vector field 
    f: array -> vector field components [Fx,Fy,Fz,...]
    sp: array -> spacing between points in respecitve directions [spx, spy,spz,...]
    """
    num_dims = len(f)
    return np.ufunc.reduce(np.add, [np.gradient(f[i], sp[i], axis=i) for i in range(num_dims)])

## Calculate interpolated fields (3D)

In [ ]:
# Initialize lists to store results
wind_fields = []
turbulent_latentheat_flux_fields = []
turbulent_sensibleheat_flux_fields = []
abshumidity_fields = []
advectedflux_fields = []
dryairdensity_fields = []
temperature_fields = []
mixingratio_fields = []

grid_spacings = []
timestamps = []


field_dataframes ={
    'wind' :            wind_field_df,              # m/s
    'turb_flux' :       turb_flux_field_df,         # g/m^2/s
    'temp_turb_flux' :  temp_turb_flux_field_df,    # K/m^2/s
    'abs_hum' :         abs_hum_field_df,           # g/m^3/s
    'adv_flux' :        adv_flux_field_df,          # g/m^2/s
    'dryair_density' :  dryair_density_field_df,    # g/m^3
    'temperature':      temp_field_df,              # ˚C
    'mixing_ratio' :    mixing_ratio_field_df,      # g/g
}

for i in wind_field_df.index:
    # sometimes we don't have all measurements - this ensures we only retrieve data that exists
    if all([i in df.index for df in field_dataframes.values()]):
        values_dataframes = {}
        
        # Isolate all measurementsfor this timestamp
        for key in field_dataframes.keys():
            values_dataframes[key] = pd.DataFrame(
                field_dataframes[key].loc[i]
            ).reset_index().set_index(['height', 'tower']).pivot(columns='measurement')
            values_dataframes[key].columns = values_dataframes[key].columns.droplevel(0)

        # Combine all measurements of fields and instrument locations into one dataframe
        points_and_fields = instrument_loc_df.join(
            values_dataframes['wind'], how='right' # join on right df, so we drop instruments that we don't have measurements for
        )
        for key in field_dataframes.keys():
            if key != 'wind':
                points_and_fields = points_and_fields.join(values_dataframes[key])

        # Create a meshgrid for the interpolation
        xx, yy, zz = np.meshgrid(
            np.linspace(points_and_fields.x.min(), points_and_fields.x.max(), HORIZ_GRID_SPACING),
            np.linspace(points_and_fields.y.min(), points_and_fields.y.max(), HORIZ_GRID_SPACING),
            np.linspace(points_and_fields.z.min(), points_and_fields.z.max(), VERT_GRID_SPACING)
        )
        points = np.transpose(np.vstack((points_and_fields.x, points_and_fields.y, points_and_fields.z)))

        # Interpolate wind field
        u_interp = interpolate.griddata(points, points_and_fields['u'], (xx, yy, zz), method='linear')
        v_interp = interpolate.griddata(points, points_and_fields['v'], (xx, yy, zz), method='linear')
        w_interp = interpolate.griddata(points, points_and_fields['w'], (xx, yy, zz), method='linear')

        # Interpolate turbulent latent heat flux fields
        u_q__interp = interpolate.griddata(points, points_and_fields['u_h2o_'], (xx, yy, zz), method='linear')
        v_q__interp = interpolate.griddata(points, points_and_fields['v_h2o_'], (xx, yy, zz), method='linear')
        w_q__interp = interpolate.griddata(points, points_and_fields['w_h2o_'], (xx, yy, zz), method='linear')

        # Interpolate turbulent sensible heat flux fields
        u_tc__interp = interpolate.griddata(points, points_and_fields['u_tc_'], (xx, yy, zz), method='linear')
        v_tc__interp = interpolate.griddata(points, points_and_fields['v_tc_'], (xx, yy, zz), method='linear')
        w_tc__interp = interpolate.griddata(points, points_and_fields['w_tc_'], (xx, yy, zz), method='linear')

        # Interpolate advected flux field
        uq_interp = interpolate.griddata(points, points_and_fields['uq'], (xx, yy, zz), method='linear')
        vq_interp = interpolate.griddata(points, points_and_fields['vq'], (xx, yy, zz), method='linear')
        wq_interp = interpolate.griddata(points, points_and_fields['wq'], (xx, yy, zz), method='linear')

        # Interpolate abs. humidity field
        q_interp = interpolate.griddata(points, points_and_fields['q'], (xx, yy, zz), method='linear')

        # Interpolate dry air density field
        rho_interp = interpolate.griddata(points, points_and_fields['rho'], (xx, yy, zz), method='linear')

        # Interpolatet temperature field
        T_interp = interpolate.griddata(points, points_and_fields['T'], (xx, yy, zz), method='linear')

        # Interpolate (water vapor) mixing ratio field
        r_interp = interpolate.griddata(points, points_and_fields['r'], (xx, yy, zz), method='linear')

        # Combine interpolated components into vector fields
        F = np.array([u_interp, v_interp, w_interp])
        Fq = np.array([uq_interp, vq_interp, wq_interp])
        F_q_ = np.array([u_q__interp, v_q__interp, w_q__interp])
        F_tc_ = np.array([u_tc__interp, v_tc__interp, w_tc__interp])

        # Record grid spacing        
        sp_x = np.diff(xx[0,:,0]).mean()
        sp_y = np.diff(yy[:,0,0]).mean()
        sp_z = np.diff(zz[0,0,:]).mean()
        sp = [sp_x, sp_y, sp_z]

        # Append interpolated fields to our results lists
        wind_fields.append(F)
        advectedflux_fields.append(Fq)
        turbulent_latentheat_flux_fields.append(F_q_)
        turbulent_sensibleheat_flux_fields.append(F_tc_)
        abshumidity_fields.append(q_interp)
        dryairdensity_fields.append(rho_interp)
        temperature_fields.append(T_interp)
        mixingratio_fields.append(r_interp)
        grid_spacings.append(sp)
        timestamps.append(i)

## Calculate interpolated fields (2D)

In [ ]:
# Initialize lists to store results
wind_fields_2d = []
turbulent_latentheat_flux_fields_2d = []
turbulent_sensibleheat_flux_fields_2d = []
abshumidity_fields_2d = []
advectedflux_fields_2d = []
dryairdensity_fields_2d = []
temperature_fields_2d = []
mixingratio_fields_2d = []

grid_spacings_2d = []
timestamps_2d = []


field_dataframes ={
    'wind' :            wind_field_df,              # m/s
    'turb_flux' :       turb_flux_field_df,         # g/m^2/s
    'temp_turb_flux' :  temp_turb_flux_field_df,    # K/m^2/s
    'abs_hum' :         abs_hum_field_df,           # g/m^3/s
    'adv_flux' :        adv_flux_field_df,          # g/m^2/s
    'dryair_density' :  dryair_density_field_df,    # g/m^3
    'temperature':      temp_field_df,              # ˚C
    'mixing_ratio' :    mixing_ratio_field_df,      # g/g
}

for i in wind_field_df.index:
    # sometimes we don't have all measurements - this ensures we only retrieve data that exists
    if all([i in df.index for df in field_dataframes.values()]):
        values_dataframes = {}
        
        # Isolate all measurementsfor this timestamp
        for key in field_dataframes.keys():
            values_dataframes[key] = pd.DataFrame(
                field_dataframes[key].loc[i]
            ).reset_index().set_index(['height', 'tower']).pivot(columns='measurement')
            values_dataframes[key].columns = values_dataframes[key].columns.droplevel(0)

        # Combine all measurements of fields and instrument locations into one dataframe
        points_and_fields = instrument_loc_df.join(
            values_dataframes['wind'], how='right' # join on right df, so we drop instruments that we don't have measurements for
        )
        for key in field_dataframes.keys():
            if key != 'wind':
                points_and_fields = points_and_fields.join(values_dataframes[key])

        # Create a meshgrid for the interpolation
        wind_fields_2d_dict = {}
        advectedflux_fields_2d_dict = {}
        turbulent_latentheat_flux_fields_2d_dict = {}
        turbulent_sensibleheat_flux_fields_2d_dict = {}
        abshumidity_fields_2d_dict = {}
        dryairdensity_fields_2d_dict = {}
        temperature_fields_2d_dict = {}
        mixingratio_fields_2d_dict = {}
        grid_spacings_2d_dict = {}
        timestamps_2d_dict = {}
        for height in HEIGHTS:
            these_points_and_fields = points_and_fields.query(f"height == {height}")
            xx, yy = np.meshgrid(
                np.linspace(these_points_and_fields.x.min(), these_points_and_fields.x.max(), HORIZ_GRID_SPACING),
                np.linspace(these_points_and_fields.y.min(), these_points_and_fields.y.max(), HORIZ_GRID_SPACING),
            )
            points = np.transpose(np.vstack((these_points_and_fields.x, these_points_and_fields.y)))

            # Interpolate wind field
            u_interp = interpolate.griddata(points, these_points_and_fields['u'], (xx, yy), method='linear')
            v_interp = interpolate.griddata(points, these_points_and_fields['v'], (xx, yy), method='linear')
            w_interp = interpolate.griddata(points, these_points_and_fields['w'], (xx, yy), method='linear')

            # Interpolate turbulent latent heat flux fields
            u_q__interp = interpolate.griddata(points, these_points_and_fields['u_h2o_'], (xx, yy), method='linear')
            v_q__interp = interpolate.griddata(points, these_points_and_fields['v_h2o_'], (xx, yy), method='linear')
            w_q__interp = interpolate.griddata(points, these_points_and_fields['w_h2o_'], (xx, yy), method='linear')

            # Interpolate turbulent sensible heat flux fields
            u_tc__interp = interpolate.griddata(points, these_points_and_fields['u_tc_'], (xx, yy), method='linear')
            v_tc__interp = interpolate.griddata(points, these_points_and_fields['v_tc_'], (xx, yy), method='linear')
            w_tc__interp = interpolate.griddata(points, these_points_and_fields['w_tc_'], (xx, yy), method='linear')

            # Interpolate advected flux field
            uq_interp = interpolate.griddata(points, these_points_and_fields['uq'], (xx, yy), method='linear')
            vq_interp = interpolate.griddata(points, these_points_and_fields['vq'], (xx, yy), method='linear')
            wq_interp = interpolate.griddata(points, these_points_and_fields['wq'], (xx, yy), method='linear')

            # Interpolate abs. humidity field
            q_interp = interpolate.griddata(points, these_points_and_fields['q'], (xx, yy), method='linear')

            # Interpolate dry air density field
            rho_interp = interpolate.griddata(points, these_points_and_fields['rho'], (xx, yy), method='linear')

            # Interpolatet temperature field
            T_interp = interpolate.griddata(points, these_points_and_fields['T'], (xx, yy), method='linear')

            # Interpolate (water vapor) mixing ratio field
            r_interp = interpolate.griddata(points, these_points_and_fields['r'], (xx, yy), method='linear')

            # Combine interpolated components into vector fields
            F = np.array([u_interp, v_interp, w_interp])
            Fq = np.array([uq_interp, vq_interp, wq_interp])
            F_q_ = np.array([u_q__interp, v_q__interp, w_q__interp])
            F_tc_ = np.array([u_tc__interp, v_tc__interp, w_tc__interp])

            # Record grid spacing        
            sp_x = np.diff(xx[0,:]).mean()
            sp_y = np.diff(yy[:,0]).mean()
            sp = [sp_x, sp_y]

            wind_fields_2d_dict[height] = F
            advectedflux_fields_2d_dict[height] = Fq
            turbulent_latentheat_flux_fields_2d_dict[height] = F_q_
            turbulent_sensibleheat_flux_fields_2d_dict[height] = F_tc_
            abshumidity_fields_2d_dict[height] = q_interp
            dryairdensity_fields_2d_dict[height] = rho_interp
            temperature_fields_2d_dict[height] = T_interp
            mixingratio_fields_2d_dict[height] = r_interp
            grid_spacings_2d_dict[height] = sp
            timestamps_2d_dict[height] = i

        # Append interpolated fields to our results lists
        wind_fields_2d.append(wind_fields_2d_dict)
        advectedflux_fields_2d.append(advectedflux_fields_2d_dict)
        turbulent_latentheat_flux_fields_2d.append(turbulent_latentheat_flux_fields_2d_dict)
        turbulent_sensibleheat_flux_fields_2d.append(turbulent_sensibleheat_flux_fields_2d_dict)
        abshumidity_fields_2d.append(abshumidity_fields_2d_dict)
        dryairdensity_fields_2d.append(dryairdensity_fields_2d_dict)
        temperature_fields_2d.append(temperature_fields_2d_dict)
        mixingratio_fields_2d.append(mixingratio_fields_2d_dict)
        grid_spacings_2d.append(grid_spacings_2d_dict)
        timestamps_2d.append(timestamps_2d_dict)

## Calculate divergences (3D)

Conservation of mass, calculated with substitution of species mass density for the product of the mixing ratio and the dry air density ($q = s \rho$). This is done so that the effects of dry air density changes caused by perturbations in temperature and water vapor (for moist air) can be separated from the effects of mixing ratio perturbations (Paw U et al., 2000).

$$
\overline{\rho} \frac{\partial \overline{s}}{\partial t} 
+ \overline{s} \frac{\partial }{\partial z} \Big( 
    \frac{\overline{\rho}}{\overline{T}} ( 1 + \mu \overline{s}) \overline{w'T'} + \mu \overline{w'q'} 
\Big)
+ \overline{u_i} \space \overline{\rho} \frac{\partial \overline{s}}{\partial x_i} 
+ \frac{\partial \overline{w'q'}}{\partial z}  = \overline{S}
$$

Continuity, with reynolds decomposition, averaging, and some assumptions, is

$$
(\overline{\rho} \frac{\partial \overline{u}}{\partial x} + \overline{\rho} \frac{\partial \overline{v}}{\partial y} + \overline{\rho} \frac{\partial \overline{w}}{\partial z}) = 
(- \overline{w} \frac{\partial \overline{\rho}}{\partial z} - \overline{u} \frac{\partial \overline{\rho} }{\partial x} - \overline{v} \frac{\partial \overline{\rho} }{\partial y})
- \frac{\partial \overline{w'\rho'}}{\partial z}
- \frac{\partial \overline{\rho} }{\partial t}
$$

and note that from Webb et al. (1980), we can substitute
$$
\frac{\partial \overline{w'\rho'}}{\partial z} = \frac{\partial}{\partial z} \biggl( 
    \frac{\overline{\rho}}{\overline{T}} (1 + \mu \overline{s} ) \overline{w'T'} + \mu \overline{w'q'})
\biggl)
$$

In [ ]:
mu = 1/0.622

In [ ]:
advective_term_lateral_ls = []
advective_term_vertical_ls = []
advective_term_total_ls = []
airdensityflux_term_vertical_ls = []
turbulent_term_vertical_ls = []

advective_density_flux_windgrad_lateral_ls = []
advective_density_flux_windgrad_vertical_ls = []
advective_density_flux_densitygrad_lateral_ls = []
advective_density_flux_densitygrad_vertical_ls = []
vertical_density_flux_deriv_term_ls = []

for i in range(0, len(wind_fields)):
    wind_field                          = wind_fields[i]
    turbulent_latentheat_flux_field     = turbulent_latentheat_flux_fields[i]
    turbulent_sensibleheat_flux_field   = turbulent_sensibleheat_flux_fields[i]
    abshumidity_field                   = abshumidity_fields[i]
    dryairdensity_field                 = dryairdensity_fields[i]
    temperature_field                  = temperature_fields[i]
    mixingratio_field                   = mixingratio_fields[i]
    grid_spacing                        = grid_spacings[i]
    
    # continuity stuff
    advective_density_flux_windgrad_lateral = (
        dryairdensity_field * np.gradient(wind_field[0], grid_spacing[0], axis=0) +
        dryairdensity_field * np.gradient(wind_field[1], grid_spacing[1], axis=1)
    )
    advective_density_flux_windgrad_vertical = (
        dryairdensity_field * np.gradient(wind_field[2], grid_spacing[2], axis=2)
    )
    advective_density_flux_densitygrad_lateral = (
        wind_field[0]*np.gradient(dryairdensity_field, grid_spacing[0], axis=0) + 
        wind_field[1]*np.gradient(dryairdensity_field, grid_spacing[1], axis=1)
    )
    advective_density_flux_densitygrad_vertical = (
        wind_field[2]*np.gradient(dryairdensity_field, grid_spacing[2], axis=2)
    )

    vertical_density_flux_term = (
            (dryairdensity_field/temperature_field) * (1 + mu*mixingratio_field) * turbulent_sensibleheat_flux_field[2] + mu*turbulent_latentheat_flux_field[2]
        )
    vertical_density_flux_deriv_term = np.gradient(
        vertical_density_flux_term,
        grid_spacing[2],
        axis=2
    )

    # conservation of moisture stuff
    ds_dx = np.gradient(mixingratio_field,  grid_spacing[0],    axis=0)
    ds_dy = np.gradient(mixingratio_field,  grid_spacing[1],    axis=1)
    ds_dz = np.gradient(mixingratio_field,  grid_spacing[2],    axis=2)
    advective_term_lateral = (
        wind_field[0] * dryairdensity_field * ds_dx + 
        wind_field[1] * dryairdensity_field * ds_dy
    )
    advective_term_vertical = (
        wind_field[2]*dryairdensity_field*ds_dz
    )
    advective_term_total = advective_term_lateral + advective_term_vertical

    airdensityflux_term_vertical = mixingratio_field * np.gradient(
        vertical_density_flux_term,
        grid_spacing[2],
        axis=2
    )

    turbulent_term_vertical = np.gradient(
        turbulent_latentheat_flux_field[2],
        grid_spacing[2],
        axis=2
    )

    advective_density_flux_windgrad_lateral_ls.append(np.nanmedian(advective_density_flux_windgrad_lateral))
    advective_density_flux_windgrad_vertical_ls.append(np.nanmedian(advective_density_flux_windgrad_vertical))
    advective_density_flux_densitygrad_lateral_ls.append(np.nanmedian(advective_density_flux_densitygrad_lateral))
    advective_density_flux_densitygrad_vertical_ls.append(np.nanmedian(advective_density_flux_densitygrad_vertical))
    vertical_density_flux_deriv_term_ls.append(np.nanmedian(vertical_density_flux_deriv_term))

    advective_term_lateral_ls.append(np.nanmedian(advective_term_lateral))
    advective_term_vertical_ls.append(np.nanmedian(advective_term_vertical))
    advective_term_total_ls.append(np.nanmedian(advective_term_total))
    airdensityflux_term_vertical_ls.append(np.nanmedian(airdensityflux_term_vertical))
    turbulent_term_vertical_ls.append(np.nanmedian(turbulent_term_vertical))

## Calculate divergences (2D)

In [ ]:
advective_term_lateral_ls_2d = []
advective_term_vertical_ls_2d = []
advective_term_total_ls_2d = []
airdensityflux_term_vertical_ls_2d = []
turbulent_term_vertical_ls_2d = []

for i in range(0, len(wind_fields_2d)):
    wind_field                          = wind_fields_2d[i]
    turbulent_latentheat_flux_field     = turbulent_latentheat_flux_fields_2d[i]
    turbulent_sensibleheat_flux_field   = turbulent_sensibleheat_flux_fields_2d[i]
    abshumidity_field                   = abshumidity_fields_2d[i]
    dryairdensity_field                 = dryairdensity_fields_2d[i]
    temperature_field                  = temperature_fields_2d[i]
    mixingratio_field                   = mixingratio_fields_2d[i]
    grid_spacing                        = grid_spacings_2d[i]
    
    advective_term_lateral_ls_2d_dict = {}  
    advective_term_vertical_ls_2d_dict = {}
    advective_term_total_ls_2d_dict = {}
    airdensityflux_term_vertical_ls_2d_dict = {}
    turbulent_term_vertical_ls_2d_dict = {}
    for height in wind_field.keys():
        ds_dx = np.gradient(mixingratio_field[height],  grid_spacing[height][0],    axis=0)
        ds_dy = np.gradient(mixingratio_field[height],  grid_spacing[height][1],    axis=1)
        ds_dz = np.gradient(
            np.stack([mixingratio_field[3], mixingratio_field[10]], axis=2),
            7,
            axis=2
        ).mean(axis=2)
        advective_term_lateral = (
            wind_field[height][0] * dryairdensity_field[height] * ds_dx + 
            wind_field[height][1] * dryairdensity_field[height] * ds_dy
        )
        advective_term_vertical = (
            wind_field[height][2]*dryairdensity_field[height]*ds_dz
        )
        advective_term_total = advective_term_lateral + advective_term_vertical

        # airdensityflux_term_vertical = mixingratio_field[height] * np.gradient(
        #     (
        #         (dryairdensity_field[height]/temperature_field[height]) * (1 + mu*mixingratio_field[height]) * turbulent_sensibleheat_flux_field[height][2] + mu*turbulent_latentheat_flux_field[height][2]
        #     ),
        #     grid_spacing[height][2],
        #     axis=2
        # )

        # turbulent_term_vertical = np.gradient(
        #     turbulent_latentheat_flux_field[height][2],
        #     grid_spacing[height][2],
        #     axis=2
        # )
        advective_term_lateral_ls_2d_dict[height] = np.nanmedian(advective_term_lateral)
        advective_term_vertical_ls_2d_dict[height] = np.nanmedian(advective_term_vertical)
        advective_term_total_ls_2d_dict[height] = np.nanmedian(advective_term_total)
        # airdensityflux_term_vertical_ls_2d_dict[height] = np.nanmedian(airdensityflux_term_vertical)
        # turbulent_term_vertical_ls_2d_dict[height] = np.nanmedian(turbulent_term_vertical)
    
    advective_term_lateral_ls_2d.append(advective_term_lateral_ls_2d_dict)
    advective_term_vertical_ls_2d.append(advective_term_vertical_ls_2d_dict)
    advective_term_total_ls_2d.append(advective_term_total_ls_2d_dict)
    # airdensityflux_term_vertical_ls_2d.append(airdensityflux_term_vertical_ls_2d_dict)
    # turbulent_term_vertical_ls_2d.append(turbulent_term_vertical_ls_2d_dict)

## Calculate spatially averaged divergence values (3D)

In [ ]:
conservation_spatial_mean_df = pd.DataFrame({
    'advective_term_lateral' : advective_term_lateral_ls,
    'advective_term_vertical' : advective_term_vertical_ls,
    'advective_term_total' : advective_term_total_ls,
    'airdensityflux_term_vertical' : airdensityflux_term_vertical_ls,
    'turbulent_term_vertical' : turbulent_term_vertical_ls,
    'dryair_density' : [np.nanmedian(item) for item in dryairdensity_fields],
    'mixing_ratio' : [np.nanmedian(item) for item in mixingratio_fields],
})
conservation_spatial_mean_df['time'] = timestamps
conservation_spatial_mean_df = conservation_spatial_mean_df.set_index('time')
conservation_spatial_mean_df['storage_change'] = conservation_spatial_mean_df['dryair_density'] * np.gradient(
    conservation_spatial_mean_df['mixing_ratio'],
    conservation_spatial_mean_df.index.diff()[1].seconds
)
conservation_spatial_mean_df['source'] = (
    conservation_spatial_mean_df['storage_change']
    + conservation_spatial_mean_df['advective_term_total']
    - conservation_spatial_mean_df['airdensityflux_term_vertical']
    + conservation_spatial_mean_df['turbulent_term_vertical']
)

continuity_spatial_mean_df = pd.DataFrame({
    'rho du/dx + rho dv/dy' : advective_density_flux_windgrad_lateral_ls,
    'rho dw/dz'             : advective_density_flux_windgrad_vertical_ls,
    'u d(rho)/dx + v drho/dy' : advective_density_flux_densitygrad_lateral_ls,
    'w d(rho)/dz'             : advective_density_flux_densitygrad_vertical_ls,
    "d(w_rho_)/dz"          : vertical_density_flux_deriv_term_ls,
    'dryair_density'        : [np.nanmedian(item) for item in dryairdensity_fields],
})
continuity_spatial_mean_df['time'] = timestamps
continuity_spatial_mean_df = continuity_spatial_mean_df.set_index('time')
continuity_spatial_mean_df['d(rho)_dt'] = np.gradient(
    continuity_spatial_mean_df['dryair_density'],
    continuity_spatial_mean_df.index.diff()[1].seconds
)

## Calculate spatially averaged divergence values (2D)

In [ ]:
conservation_spatial_mean_2d_df = pd.DataFrame({
    'advective_term_lateral_1m' : [d[3] for d in advective_term_lateral_ls_2d],
    'advective_term_lateral_3m' : [d[3] for d in advective_term_lateral_ls_2d],
    'advective_term_lateral_10m' : [d[10] for d in advective_term_lateral_ls_2d],
    'advective_term_vertical_3m' : [d[3] for d in advective_term_vertical_ls_2d],
    'advective_term_vertical_10m' : [d[10] for d in advective_term_vertical_ls_2d],
    'advective_term_total_3m' : [d[3] for d in advective_term_total_ls_2d],
    'advective_term_total_10m' : [d[10] for d in advective_term_total_ls_2d],
    # 'advective_term_total_1m' : [d[1] for d in advective_term_total_ls_2d],
    # 'advective_term_total_3m' : [d[3] for d in advective_term_total_ls_2d],
    # 'advective_term_total_10m' : [d[10] for d in advective_term_total_ls_2d],
    # 'airdensityflux_term_vertical' : airdensityflux_term_vertical_ls,
    # 'turbulent_term_vertical' : turbulent_term_vertical_ls,
    # 'dryair_density' : [np.nanmedian(item) for item in dryairdensity_fields],
    # 'mixing_ratio' : [np.nanmedian(item) for item in mixingratio_fields],
})
conservation_spatial_mean_2d_df['time'] = [d[10] for d in timestamps_2d]
conservation_spatial_mean_2d_df = conservation_spatial_mean_2d_df.set_index('time')

# Explore how interpolation method affects divergence calculations

In [ ]:
interpolate.Rbf(
    points_and_fields[['x','y','z','r']].dropna().x,
    points_and_fields[['x','y','z','r']].dropna().y,
    points_and_fields[['x','y','z','r']].dropna().z,
    points_and_fields[['x','y','z','r']].dropna().r
)(
    np.linspace(points_and_fields.x.min(), points_and_fields.x.max(), HORIZ_GRID_SPACING),
    np.linspace(points_and_fields.y.min(), points_and_fields.y.max(), HORIZ_GRID_SPACING),
    np.linspace(points_and_fields.z.min(), points_and_fields.z.max(), HORIZ_GRID_SPACING)
)


interpolate.Rbf(
    points_and_fields[['x','y','z','r']].dropna().x,
    points_and_fields[['x','y','z','r']].dropna().y,
    points_and_fields[['x','y','z','r']].dropna().z,
    points_and_fields[['x','y','z','r']].dropna().r
)(
    np.linspace(points_and_fields.x.min(), points_and_fields.x.max(), HORIZ_GRID_SPACING),
    np.linspace(points_and_fields.y.min(), points_and_fields.y.max(), HORIZ_GRID_SPACING),
    np.linspace(points_and_fields.z.min(), points_and_fields.z.max(), HORIZ_GRID_SPACING)
)


In [ ]:
xx_horiz, yy_horiz = np.meshgrid(
    np.linspace(points_and_fields.x.min(), points_and_fields.x.max(), HORIZ_GRID_SPACING),
    np.linspace(points_and_fields.y.min(), points_and_fields.y.max(), HORIZ_GRID_SPACING),
)

src_linear = interpolate.griddata(
    points_and_fields.loc[3][['x','y']].values, 
    points_and_fields.loc[3]['r'], 
    (xx_horiz, yy_horiz), 
    method='linear'
)
src_cubic = interpolate.griddata(
    points_and_fields.loc[3][['x','y']].values, 
    points_and_fields.loc[3]['r'], 
    (xx_horiz, yy_horiz), 
    method='cubic'
)
src_grad_linear = (
    np.gradient(src_linear, grid_spacing[0], axis=0) + 
    np.gradient(src_linear, grid_spacing[1], axis=1)
)
src_grad_cubic = (
    np.gradient(src_cubic, grid_spacing[0], axis=0) + 
    np.gradient(src_cubic, grid_spacing[1], axis=1)
)
fig, axes = plt.subplots(2,2, sharex=True, sharey=True)
axes[0,0].imshow(src_linear)
axes[0,1].imshow(src_cubic)
axes[1,0].imshow(src_grad_linear)
axes[1,1].imshow(src_grad_cubic)
axes[0,0].set_title("linear interp.\nq")
axes[0,1].set_title("cubic interp.\nq")
axes[1,0].set_title(f"dq/dx + dq/dy \n(median = {round(np.nanmedian(src_grad_linear),8)})")
axes[1,1].set_title(f"dq/dx + dq/dy \n(median = {round(np.nanmedian(src_grad_cubic),8)})")
plt.tight_layout()

# Plot results

In [ ]:
line = alt.Chart().mark_rule().encode(y=alt.datum(0))

def conservation_spatial_mean_composite_plot(src, columns, title, times_filter = None, normalize = None):
    src = src.copy(deep=True)
    if times_filter is not None:
        src = src[src.time.isin(times_filter)]
    else:
        src = src
    if normalize is not None:
        src[columns] = src[columns] * normalize
    return line + alt.Chart(
        src.reset_index()
    ).transform_fold(
        columns
    ).mark_line().encode(
        alt.X('hoursminutes(time):T').title('time'),
        alt.Y('median(value):Q'),
        alt.Color('key:N')
    ).properties(height = 200, width = 200, title=title)

In [ ]:
method_3d_chart = conservation_spatial_mean_composite_plot(
        conservation_spatial_mean_df,
        [
            'advective_term_lateral', 'advective_term_vertical', 'advective_term_total', 
            'airdensityflux_term_vertical', 'turbulent_term_vertical', 'storage_change', 'source'
        ],
        title='3D Method', normalize = 7
    )
method_2d_chart = conservation_spatial_mean_composite_plot(
        conservation_spatial_mean_2d_df, 
        [c for c in list(conservation_spatial_mean_2d_df.columns) if '1m' not in c], 
        '2D Method, cubic interpolation', 
        times_filter = None, 
        normalize = 7
    )

In [ ]:
src = conservation_spatial_mean_df.join(
    tidy_df.query("variable=='w_h2o__3m_c'")[['time', 'value']].set_index('time').rename(columns={'value':'w_h2o__3m_c'})
)
alt.Chart(
    src.reset_index()
).transform_fold(
    ['advective_term_lateral', 'w_h2o__3m_c', 'advective_term_vertical']
).mark_line().encode(
    alt.X('hoursminutes(time):T'),
    alt.Y('median(value):Q').title('Water vapor flux density (g/m^2/s)').scale(
        domain=[-0.001, 0.006]
    ),
    alt.Color('key:N').sort([
        'w_h2o__3m_c', 'advective_term_lateral', 'advective_term_vertical'
    ])
    
).properties(width = 200, height = 200).display(renderer='svg')

In [ ]:
method_3d_chart = conservation_spatial_mean_composite_plot(
        conservation_spatial_mean_df,
        [
            'advective_term_lateral', 
            'advective_term_vertical', 
            'advective_term_total', 
            # 'airdensityflux_term_vertical', 'turbulent_term_vertical', 'storage_change', 'source'
        ],
        title='3D Method', normalize = 7
    )

In [ ]:
alt.Chart(
    conservation_spatial_mean_df.loc['20221101':'20221120'].reset_index()
).transform_fold([
    'advective_term_lateral', 
    'advective_term_vertical', 
    'advective_term_total', 
]).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q').scale(domain=[-0.1,0.1], clamp=True),
    alt.Color('key:N')
).properties(width=800)

In [ ]:
conservation_spatial_mean_composite_plot(
        conservation_spatial_mean_df.loc['20221101':'20221120'],
        [
            'advective_term_lateral', 
            'advective_term_vertical', 
            'advective_term_total', 
            # 'airdensityflux_term_vertical', 'turbulent_term_vertical', 'storage_change', 'source'
        ],
        title='3D Method', normalize = 7
    ) | conservation_spatial_mean_composite_plot(
        conservation_spatial_mean_df.loc['20230601':'20230619'],
        [
            'advective_term_lateral', 
            'advective_term_vertical', 
            'advective_term_total', 
            # 'airdensityflux_term_vertical', 'turbulent_term_vertical', 'storage_change', 'source'
        ],
        title='3D Method', normalize = 7
    )

In [ ]:
src = continuity_spatial_mean_df.drop(columns='dryair_density')
src['conservation'] = src.sum(axis=1)
alt.Chart(src.reset_index()).transform_fold(
    list(src.columns)
).mark_line().encode(
    alt.X('hoursminutes(time):T').title('time'),
    alt.Y('median(value):Q'),
    alt.Color('key:N', legend=alt.Legend(labelLimit=1000))
)

In [ ]:
src = tidy_df.query("measurement == 'wind speed'").query("tower == 'c'").set_index(['time', 'height'])
src = src.groupby([
    src.index.get_level_values(0).floor('120T').time, 'height'
])[['value']].mean()
src = src.reset_index().rename(columns={'level_0':'time'})
src['hour'] = src.time.apply(lambda t: t.hour)
chart = alt.Chart(
    src
).mark_line().encode(
    alt.X('value:Q').sort('-y'),
    alt.Y('height:Q'),
    alt.Color('hour:O').scale(scheme='rainbow')
).properties(width=110,height=110)
(
    chart.transform_filter(alt.FieldOneOfPredicate('hour', [0, 2,4,6])) |\
    chart.transform_filter(alt.FieldOneOfPredicate('hour', [8,10,12,14])) |\
    chart.transform_filter(alt.FieldOneOfPredicate('hour', [16,18,20,22]))
).configure_legend(orient='top').resolve_scale(color='shared')

In [ ]:
src = tidy_df.query("measurement == 'wind speed'").query("tower == 'c'")
src = src.set_index('time').loc['20230203':'20230203'].reset_index()
src = src.set_index(['time', 'height'])
src = src.groupby([
    src.index.get_level_values(0).floor('120T').time, 'height'
])[['value']].mean()
src = src.reset_index().rename(columns={'level_0':'time'})
src['hour'] = src.time.apply(lambda t: t.hour)
chart = alt.Chart(
    src
).mark_line().encode(
    alt.X('value:Q').sort('-y'),
    alt.Y('height:Q'),
    alt.Color('hour:O').scale(scheme='rainbow')
).properties(width=110,height=110)
(
    chart.transform_filter(alt.FieldOneOfPredicate('hour', [0, 2,4,6])) |\
    chart.transform_filter(alt.FieldOneOfPredicate('hour', [8,10,12,14])) |\
    chart.transform_filter(alt.FieldOneOfPredicate('hour', [16,18,20,22]))
).configure_legend(orient='top').resolve_scale(color='shared')

In [ ]:
src = tidy_df.query("measurement == 'wind direction'")
src = src.set_index('time').loc['20230203':'20230203'].reset_index()
src = src.set_index(['time', 'height', 'tower'])
src = src.groupby([
    src.index.get_level_values(0).floor('120T').time, 'height', 'tower'
])[['value']].mean()
src = src.reset_index().rename(columns={'level_0':'time'})
src['hour'] = src.time.apply(lambda t: t.hour)
chart = alt.Chart(
    src
).mark_line().encode(
    alt.X('value:Q').sort('-y'),
    alt.Y('height:Q'),
    alt.Color('hour:O').scale(scheme='rainbow'),
    alt.StrokeDash('tower:N')
).properties(width=110,height=110)
(
    chart.transform_filter(alt.FieldOneOfPredicate('hour', [0, 2,4,6])) |\
    chart.transform_filter(alt.FieldOneOfPredicate('hour', [8,10,12,14])) |\
    chart.transform_filter(alt.FieldOneOfPredicate('hour', [16,18,20,22]))
).configure_legend(orient='top').resolve_scale(x='shared', color='shared').display(renderer='svg')

In [ ]:
src = tidy_df.query("measurement == 'wind speed'")
src = src.set_index('time').loc['20230203':'20230203'].reset_index()
src = src.set_index(['time', 'height', 'tower'])
src = src.groupby([
    src.index.get_level_values(0).floor('120T').time, 'height', 'tower'
])[['value']].mean()
src = src.reset_index().rename(columns={'level_0':'time'})
src['hour'] = src.time.apply(lambda t: t.hour)
chart = alt.Chart(
    src
).mark_line().encode(
    alt.X('value:Q').sort('-y'),
    alt.Y('height:Q'),
    alt.Color('hour:O').scale(scheme='rainbow'),
    alt.StrokeDash('tower:N')
).properties(width=110,height=110)
(
    chart.transform_filter(alt.FieldOneOfPredicate('hour', [0, 2,4,6])) |\
    chart.transform_filter(alt.FieldOneOfPredicate('hour', [8,10,12,14])) |\
    chart.transform_filter(alt.FieldOneOfPredicate('hour', [16,18,20,22]))
).configure_legend(orient='top').resolve_scale(color='shared').display(renderer='svg')

In [ ]:
(
    conservation_spatial_mean_composite_plot(
        conservation_spatial_mean_df.reset_index(),
        [
            'advective_term_lateral', 'advective_term_vertical', 'advective_term_total', 
            'airdensityflux_term_vertical', 'turbulent_term_vertical', 'storage_change', 'source'
        ],
        title='All data', normalize = 7
    ) | conservation_spatial_mean_composite_plot(
        conservation_spatial_mean_df.reset_index(),
        [
            'advective_term_lateral', 'advective_term_vertical', 'advective_term_total', 
            'airdensityflux_term_vertical', 'turbulent_term_vertical', 'storage_change', 'source'
        ],
        title='No BS', normalize = 7,
        times_filter = set(nobs_times)
    ) | conservation_spatial_mean_composite_plot(
        conservation_spatial_mean_df.reset_index(),
        [
            'advective_term_lateral', 'advective_term_vertical', 'advective_term_total', 
            'airdensityflux_term_vertical', 'turbulent_term_vertical', 'storage_change', 'source'
        ],
        title='No BS, not snowing', normalize = 7,
        times_filter = set(nobs_times).intersection(set(is_not_snowing_dates))
    ) | conservation_spatial_mean_composite_plot(
        conservation_spatial_mean_df.reset_index(),
        [
            'advective_term_lateral', 'advective_term_vertical', 'advective_term_total', 
            'airdensityflux_term_vertical', 'turbulent_term_vertical', 'storage_change', 'source'
        ],
        title='Not snowing', normalize = 7,
        times_filter = set(is_not_snowing_dates)
    ) | conservation_spatial_mean_composite_plot(
        conservation_spatial_mean_df.reset_index(),
        [
            'advective_term_lateral', 
            'advective_term_vertical', 
            'advective_term_total', 
            'airdensityflux_term_vertical', 
            'turbulent_term_vertical', 
            'storage_change', 'source'
        ],
        title='BS', normalize = 7,
        times_filter = set(bs_times)
    )
).resolve_scale(y='shared', x='shared', color='shared').display(renderer='svg')

In [ ]:
(
    conservation_spatial_mean_composite_plot(
        conservation_spatial_mean_df,
        [
            'advective_term_lateral', 'advective_term_vertical', 'advective_term_total', 
            'airdensityflux_term_vertical', 'turbulent_term_vertical', 'storage_change', 'source'
        ],
        title='All data', normalize = 7
    ) | conservation_spatial_mean_composite_plot(
        conservation_spatial_mean_df,
        [
            'advective_term_lateral', 'advective_term_vertical', 'advective_term_total', 
            'airdensityflux_term_vertical', 'turbulent_term_vertical', 'storage_change', 'source'
        ],
        title='Stable, No BS', normalize = 7,
        times_filter = set(tgrad_stable_times).intersection(set(nobs_times))
    ) | conservation_spatial_mean_composite_plot(
        conservation_spatial_mean_df,
        [
            'advective_term_lateral', 'advective_term_vertical', 'advective_term_total', 
            'airdensityflux_term_vertical', 'turbulent_term_vertical', 'storage_change', 'source'
        ],
        title='Neutral, No BS', normalize = 7,
        times_filter = set(tgrad_neutral_times).intersection(set(nobs_times))
    ) | conservation_spatial_mean_composite_plot(
        conservation_spatial_mean_df,
        [
            'advective_term_lateral', 'advective_term_vertical', 'advective_term_total', 
            'airdensityflux_term_vertical', 'turbulent_term_vertical', 'storage_change', 'source'
        ],
        title='Unstable, No BS', normalize = 7,
        times_filter = set(tgrad_unstable_times).intersection(set(nobs_times))
    )
).resolve_scale(y='shared', x='shared', color='shared').display(renderer='svg')

In [ ]:
(
    conservation_spatial_mean_composite_plot(
        conservation_spatial_mean_df,
        [
            'advective_term_lateral', 'advective_term_vertical', 'advective_term_total', 
            'airdensityflux_term_vertical', 'turbulent_term_vertical', 'storage_change',	'source'
        ],
        title='Flux density terms (g/m^2/s)', normalize = 7,
        times_filter = set(nobs_times)
    ) 
    | 
    alt.Chart(
        tidy_df[
            tidy_df.variable.isin(['w_h2o__3m_c', 'w_h2o__5m_c', 'w_h2o__10m_c'])
            &
            tidy_df.time.isin(set(nobs_times))
        ]
    ).mark_line().encode(
        alt.X('hoursminutes(time):T'),
        alt.Y('median(value):Q'),
        alt.Color('height:O')
    ).properties(
        width = 200, height = 200, title=('Vertical eddy covariance flux (g/m^2/s)')
    )
).resolve_scale(y='shared', color='independent').configure_legend(orient='top', columns=2).display(renderer='svg')

In [ ]:

src1 = conservation_spatial_mean_df.set_index('time').loc['20221221': '20221223'].reset_index().assign(casestudy = 1)
src2 = conservation_spatial_mean_df.set_index('time').loc['20221212': '20221214'].reset_index().assign(casestudy = 2)
src3 = conservation_spatial_mean_df.set_index('time').loc['20230201': '20230203'].reset_index().assign(casestudy = 3)
src = pd.concat([src1, src2, src3])
alt.Chart(src).transform_fold([
    'advective_term_lateral', 'advective_term_vertical',
    'advective_term_total', 'airdensityflux_term_vertical',
    'turbulent_term_vertical', 'storage_change', 'source',
]).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q').scale(domain=[-0.04,0.04], clamp=True),
    alt.Color('key:N'),
    alt.Facet('casestudy:N').title(None)
).properties(width = 300, height = 200).resolve_scale(x='independent').display(renderer='svg')

In [ ]:
alt.Chart(
    conservation_spatial_mean_df.set_index('time').loc['20230401': '20230407'].reset_index()
).transform_fold([
    'advective_term_lateral', 'advective_term_vertical',
    'advective_term_total', 'airdensityflux_term_vertical',
    'turbulent_term_vertical', 'storage_change', 'source',
]).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q').scale(domain=[-0.04,0.04], clamp=True),
    alt.Color('key:N'),
    alt.Facet('casestudy:N').title(None)
).properties(width =600, height = 200).resolve_scale(x='independent').display(renderer='svg')

In [ ]:
(
    (
        alt.Chart(
            (conservation_spatial_mean_df.set_index('time') * 7).loc['20230202': '20230204'].reset_index().assign(casestudy = 3)
        ).transform_fold([
            'advective_term_lateral', 
            'advective_term_vertical',
            # 'advective_term_total', 
            'airdensityflux_term_vertical',
            'turbulent_term_vertical', 
            # 'storage_change', 
            # 'source',
        ]).mark_line().encode(
            alt.X('time:T').axis(None),
            alt.Y('value:Q').scale(domain=[-0.1,0.1], clamp=True).title('Flux density (g/m^2/s)'),
            alt.Color('key:N'),
            alt.Facet('casestudy:N').title(None)
        ).properties(width = 800, height = 200) &\
        alt.Chart(
            tidy_df.query("variable == 'w_h2o__3m_c'").set_index('time').loc['20230202': '20230204'].reset_index()
        ).mark_line().encode(
            alt.X('time:T').axis(None),
            alt.Y('value:Q').title(["Vertical eddy cov.", "flux density (g/m^2/s)"]),
            alt.Color('height:O')
        ).properties(width = 800, height = 100)
     ).resolve_scale(y='shared') &\
    alt.Chart(
        tidy_df.query("measurement == 'wind direction'").query("tower == 'c'").query(
            "height <= 10"
        ).set_index('time').loc['20230202': '20230204'].reset_index()
    ).mark_line().encode(
        alt.X('time:T').axis(None),
        alt.Y('value:Q').title('Wind Dir. (˚)'),
        alt.Color('height:O')
    ).properties(width = 800, height = 100) &\
    alt.Chart(
        tidy_df.query("variable == 'Ri_3m_c'").set_index('time').loc['20230202': '20230204'].reset_index()
    ).mark_line().encode(
        alt.X('time:T').axis(None),
        alt.Y('value:Q').scale(type='symlog', domain=[-10,10], clamp=True).title('Ri (at 3 meters)'),
        alt.Color('height:O')
    ).properties(width = 800, height = 100) &\
    alt.Chart(
        tidy_df.query("variable == 'spd_10m_c'").set_index('time').loc['20230202': '20230204'].reset_index()
    ).mark_line().encode(
        alt.X('time:T'),
        alt.Y('value:Q').title('Wind speed (m/s)'),
        alt.Color('height:O')
    ).properties(width = 800, height = 100)
).resolve_scale(color='independent')

In [ ]:
alt.Chart(
    tidy_df.query("measurement == 'wind direction'").query("tower == 'c'").query(
        "height <= 10"
    ).set_index('time').loc['20221221': '20221223'].reset_index()
).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('height:O')
).properties(width = 300, height = 200) | alt.Chart(
    tidy_df.query("measurement == 'wind direction'").query("tower == 'c'").query(
        "height <= 10"
    ).set_index('time').loc['20221212': '20221214'].reset_index()
).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('height:O')
).properties(width = 300, height = 200) | alt.Chart(
    tidy_df.query("measurement == 'wind direction'").query("tower == 'c'").query(
        "height <= 10"
    ).set_index('time').loc['20230201': '20230203'].reset_index()
).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('height:O')
).properties(width = 300, height = 200)

In [ ]:
src = tidy_df[
    tidy_df.time.isin(
        set(nobs_times).intersection(set(is_not_snowing_dates))
    )
]
src = src[src.variable.isin(['w_h2o__3m_c', 'w_h2o__5m_c', 'w_h2o__10m_c'])]
alt.Chart(
    src
).mark_line().encode(
    alt.X('hoursminutes(time):T'),
    alt.Y('median(value):Q'),
    alt.Color('height:O')
).properties(height = 200, width = 200, title='Vertical turbulent flux (g/m^2/s)')

In [ ]:
alt.Chart(
    tidy_df.query("measurement == 'air density'").query("height > 0")
).mark_line().encode(
    alt.X('hoursminutes(time):T'),
    alt.Y('median(value):Q').scale(zero=False),
    alt.Color('height:O')
) | alt.Chart(
    tidy_df.query("measurement == 'temperature'").query("height > 0")
).mark_line().encode(
    alt.X('hoursminutes(time):T'),
    alt.Y('median(value):Q').scale(zero=False),
    alt.Color('height:O')
)

In [ ]:
src = mixing_ratio_field_df.copy()
src.columns = src.columns.droplevel(2)
src.columns = [f"{int(c1)}_{c2}" for (c1,c2) in zip(
    src.columns.get_level_values(0),
    src.columns.get_level_values(1)
)]
src = src.melt(ignore_index=False)
src['height'] = src['variable'].apply(lambda s: int(s.split('_')[0]))
src['tower'] = src['variable'].apply(lambda s: s.split('_')[1])
src = src[src.tower != 'c']
daily_mixingratio_chart = alt.Chart(src.reset_index()).mark_line().encode(
    alt.X('hoursminutes(time):T'),
    alt.Y('median(value):Q').scale(zero=False),
    alt.Color('tower:N'),
    alt.StrokeDash('height:O'),
).properties(title='Daily composite water vapor mixing ratio (g/g)')

In [ ]:
src = mixing_ratio_field_df.copy()
src.columns = src.columns.droplevel(2)
src.columns = [f"{int(c1)}_{c2}" for (c1,c2) in zip(
    src.columns.get_level_values(0),
    src.columns.get_level_values(1)
)]
src = src.melt(ignore_index=False)
src['height'] = src['variable'].apply(lambda s: int(s.split('_')[0]))
src['tower'] = src['variable'].apply(lambda s: s.split('_')[1])
src = src.reset_index()
src = src[(src.time > '20230201') & (src.time < '20230205')]
src = src.query("height == 3")
src.to_csv('irga_mixingratio_estimates.csv')
alt.Chart(src).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('tower:N')
).properties()

In [ ]:
src = mixing_ratio_field_df.copy()
src.columns = src.columns.droplevel(2)
src.columns = [f"{int(c1)}_{c2}" for (c1,c2) in zip(
    src.columns.get_level_values(0),
    src.columns.get_level_values(1)
)]
src = src.melt(ignore_index=False)
src['height'] = src['variable'].apply(lambda s: int(s.split('_')[0]))
src['tower'] = src['variable'].apply(lambda s: s.split('_')[1])
src = src.reset_index()
src = src[(src.time > '20221201') & (src.time < '20221205')]
src = src.query("height == 3")
src.to_csv('irga_mixingratio_estimates_2.csv')
alt.Chart(src).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('tower:N')
).properties()

In [ ]:
src

In [ ]:
src = dryair_density_field_df.copy()
src.columns = src.columns.droplevel(2)
src.columns = [f"{int(c1)}_{c2}" for (c1,c2) in zip(
    src.columns.get_level_values(0),
    src.columns.get_level_values(1)
)]
src = src.melt(ignore_index=False)
src['height'] = src['variable'].apply(lambda s: int(s.split('_')[0]))
src['tower'] = src['variable'].apply(lambda s: s.split('_')[1])
src = src.reset_index()
src = src[(src.time > '20230201') & (src.time < '20230205')]
src = src.query("height == 3")
alt.Chart(src).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q').scale(zero=False),
    alt.Color('tower:N')
).properties()

In [ ]:
tc_field_df = tidy_df[tidy_df.measurement.isin(['virtual temperature']) & tidy_df.height.isin(HEIGHTS)]
tc_field_df.measurement = 'tc'
tc_field_df = round(
    tc_field_df.pivot_table(
        index='time', columns=['height', 'tower', 'measurement'], values='value'
    ), 
    4
)

src = tc_field_df.copy()
src.columns = src.columns.droplevel(2)
src.columns = [f"{int(c1)}_{c2}" for (c1,c2) in zip(
    src.columns.get_level_values(0),
    src.columns.get_level_values(1)
)]
src = src.melt(ignore_index=False)
src['height'] = src['variable'].apply(lambda s: int(s.split('_')[0]))
src['tower'] = src['variable'].apply(lambda s: s.split('_')[1])
src = src[src.tower != 'c']
daily_sonic_temp_chart = alt.Chart(src.reset_index()).mark_line().encode(
    alt.X('hoursminutes(time):T'),
    alt.Y('median(value):Q').scale(zero=False
                                   ),
    alt.Color('tower:N'),
    alt.StrokeDash('height:O'),
).properties(title='Daily composite sonic temperature (˚C)')

In [ ]:
(daily_mixingratio_chart | daily_sonic_temp_chart
).display(renderer='svg')